<a href="https://colab.research.google.com/github/dauten/DeepLearning/blob/develop/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function

import time
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [0]:
'''
Given an array of training data and an array of labels, return a model that fits
that data within (currently hardcoded) parameters

data should be a list of char array's, labels should be a numeric identifier or quality
'''
def train(data, labels, file):
  print(tf.__version__)
  N = 0 #This is the item length, we need to find number of input edges
  for each in data:
    if len(each) > N:
      N = len(each)

  M = 0 #This is the item length, we need to find number of input edges
  for each in labels:
    if each > M:
      M = each

  #this does more processing to the image, cconverting it from a 28x28 matrix to a 784 length array.
  #the dense commands actually create neurons/nodes
  model = keras.Sequential([keras.layers.Flatten(input_shape=((N,))),
      keras.layers.Dense(256, activation=tf.nn.relu), keras.layers.Dense(256, activation=tf.nn.relu), keras.layers.Dense(256, activation=tf.nn.relu),
      keras.layers.Dense(M+1, activation=tf.nn.softmax)])

  #this tells the neural net how to update things for training
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


  #convert data and labels to ndarrays D and L
  train_images = np.ndarray((len(data), N))
  train_labels = np.ndarray((len(labels)))
  k = 0
  for i in data:
      q = 0
      for j in i:
        train_images[k][q] = ord(j)
        q+=1
      k+=1

  k = 0
  for l in labels:
      print(l)
      train_labels[k] = l
      k+=1

  print("Images:")
  print(train_images)
  print("Labels:")
  print(train_labels)

  #this is the actual 'train' step
  #you can play with epochs, increasing it takes longer but provides slightly more accuracy
  model.fit(train_images, train_labels, epochs=50)

  print(model.predict(train_images))

  model.save(file)

  return N

In [0]:
'''
Given a set of test data, return predictions from the (currently hardcoded) model
'''
def evaluate(test,pad,file):

  N = 0 #This is the item length, we need to find number of input edges
  for each in test:
    if len(each) > N:
      N = len(each)

  #convert data and labels to ndarrays D and L
  train_images = np.ndarray((len(test), pad))
  k = 0
  for i in test:
      q = 0
      for j in i:
        train_images[k][q] = ord(j)
        q+=1
      k+=1
  model =  keras.models.load_model(file)

  print(model.predict(train_images))



In [0]:
def fin(filedir):
  #give a filedirectory, go through it to return tuple of data and labels
  #should consider each folder in filedir/ to be a new identifier (so len(labels)->150)
  #with the content of each file being the data
  return (None, None) #(data, labels)

In [0]:
print("starting:")
pad = train([list("dear bob, this is jason"),list("this is also jason"),list("this is bob"), list("Hi Jason And BoB"), list("It Is Me"), list("Still ME"), list("NoT TeM BuT ME"), list("bob"), list("b"), list("hello peeps its jason")], [0,0,1,2, 2, 2, 2, 1, 1,0], "placeholder.h5")

In [318]:
#A note on the lengths: I seem to be getting bad results when N is much larger than the avg message size!
print("evaluating: (the super hot matix should be 0, 2, 0, 0, 1, 1)")
evaluate([list("w/y doing signed jason"), list("it is me signed me"), list("dear bob, still jason"),list("yet again I'm jason"),list("me is bob"),list("thii is bob")], pad, "placeholder.h5")

evaluating: (the super hot matix should be 0, 2, 0, 0, 1, 1)
[[1.0000000e+00 2.1678396e-23 8.6541566e-29]
 [          nan           nan           nan]
 [1.0000000e+00 1.1891095e-26 0.0000000e+00]
 [1.0000000e+00 2.8520799e-18 7.2779111e-31]
 [0.0000000e+00 2.9431720e-13 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00 2.0052928e-21]]


In [0]:
#once fin is implemented and the files are in place this block should
#work to train and evaluate our NN :)
data,labels = fin("enron_train/")
pad = train(data, labels, "enron.h5")
data,labels = fin("enron_test/")
evaluate(data, pad, "enron.h5")